# CHAPTER3 イベントとフォーム入力の受け取り


## 3.1 イベントハンドリング
`v-on`ディレクティブの話. addEventListenerやjQueryのonメソッドと同じ.

### メソッドイベントハンドラ

メソッドイベントハンドラでは、`v-on`に`methods`で定義したメソッド名(キー)を指定する.

```html
<div id="app">
  <button v-on:click="addCount">Add 1</button>
  <!--
    簡単なものなら、v-on:click="counter += 1"のように直接式を書くこともできる.
    [インラインメソッドハンドラ]と呼ばれる.
  -->
  <p>The button above has been clicked {{ counter }} times.</p>
</div>
```

```js
var app = new Vue({
  el: '#app',
  data: {
    counter: 0
  },
    methods: {
        addCount: function() {
            this.counter += 1
        }
    }
})
```

* `v-on`は`@`で省略できる.
```html
<button @click="addCount">Add 1</button>
```

* ブラウザJSのaddEventListenerなどで受け取れる`event`オブジェクトは、Vueのイベントハンドラでは`$event`で受け取れる.
```html
<button @click="addCount($event)">Add 1</button>
```

```js
new Vue({
    el: '#app',
    methods: {
        addCount: function(event) {
            event.preventDefault()
            event.stopPropagation()
            ... //各種eventオブジェクトのプロパティやメソッドを使うことができる
        }
    }
})
```

### 使用可能なイベント

`v-on`で利用できるイベントはブラウザに実装されているイベントと同じものが使える。
* 言い換えれば、`v-on`で利用できるイベントはブラウザの仕様に依存する.
```html
<img v-on:load="doSomething"> <!-- 画像の読み込みが完了したらdoSomethingを実行 -->
```

このほかにも`scroll Event`を扱う`v-on:scroll`や、`マウスホイールイベント`を扱う`v-on:mousewheel`などがある.

### フォーム入力の取得
`input`要素の入力値`input.value`の取得に関して、Vue.jsでは`v-model`ディレクティブを使う方法だけでなく、`v-on:input`を使ったフック処理で行うことも可.

```html
<input type="text" v-model="message">
<input type="text" v-on:input="handleInput">
```
```js
new Vue({
    el: '#app',
    data: {
        message: ''
    },
    methods: {
        handleInput: function(event) {
            this.message = event.target.value //素のJSでの処理と同じ
        }
    }
});
```

### event修飾子

clickイベントやloadイベントなどで使われる`event.preventDefault()`や`event.stopPropagation()`を、`v-on`ディレクティブの修飾子として使うことができる.

```html
<a v-on:click.prevent="doSomething">preventDefault()</a>
<a v-on:click.stop="doSomething">stopPropagation()</a>
<a v-on:click.self="doSomething">イベントの発生がこの要素自身の場合のみ実行</a>
<a v-on:click.capture="doSomething">キャプチャーモードでDOMイベントをハンドル</a>
```
* `@click`のみ、マウスのボタンを識別する修飾子が使える.
```html
<a @click.left="doSomething">左クリック</a>
<a @click.right="doSomething">右クリック</a>
<a @click.middle="doSomething">ホイールクリック</a>
```

### keyイベントの修飾子
Vue.jsでは、キーボードイベントを修飾子として、特定のキーのみを対象にイベントを発火させることができる.

```html
<input type="text" v-on:keyup.enter="doSomething"> <!-- エンターキーを押したときにdoSomethingを実行 -->
<!--以下のものはすでにVue.jsで定義されている
    .enter : Enterキー
    .tab : Tabキー
    .delete (捕捉されない場合がある) : DeleteキーとBackspaceキー
    .esc : Escキー
    .space : Spaceキー
    .up : 上矢印キー
    .down : 下矢印キー
    .left : 左矢印キー
    .right : 右矢印キー
-->
```
複数のキー修飾子を連鎖させた場合、ORの関係になる.
```html
<input type="text" v-on:keyup.enter.space="doSomething"> <!-- エンターキーまたはスペースキーを押したときにdoSomethingを実行 -->
```

### フォーム入力バインディング

フォームの入力値とデータを同期させることを`双方向バインディング`という.
双方向バインディングを行うことで、「いちいちDOMから入力値をとってきて、それをDOMのtextContentに反映して、、、」といった一連の処理をVue.jsが自動で行ってくれる.
```html
<input type="text" v-model="message">
<p>{{ message }}</p>
<!--
    messageの値が変わると、inputのvalueも変わる->inputのvalueが変わると、messageの値も変わる
-->
```
```js
new Vue({
    el: '#app',
    data: {
        message: ''
    }
});
```

* `<textarea></textarea>`の場合は、`{{}}`を使った双方向バインディングはできない.
```html
<textarea v-model="message"></textarea>
<p>{{ message }}</p> <!-- これは表示されない -->
```
`v-model`, `v-bind`を使って同期させる。
```html
<textarea v-model="message"></textarea>
<p v-model="message"></p>
```

* `v-model`の修飾子
```html
<input type="text" v-model.lazy="message"> <!-- 入力が完了したときにデータを更新する -->
<input type="text" v-model.number="age"> <!-- 入力値を数値に変換してデータを更新する -->
<input type="text" v-model.trim="message"> <!-- 入力値の両端の空白を削除してデータを更新する -->
```


## 3.15 マウント要素以外のイベントと動作

コンポーネントをマウントしている要素以外Vue.jsは動作しない。
したがって、`<body>`や`window`オブジェクトには`v-on`によるハンドルはできない。
これまでどおり、`addEventListener`, `removeEventListener`を使う必要がある。
